# Version 2.5: 중저가 예약 매장 요약문 생성 (배치 모드)

이 노트북은 **리뷰 데이터를 사용하지 않고**, 외부 정보만으로 중저가 예약 매장의 요약문을 **일괄 생성**합니다.

## 📌 핵심 특징

### 데이터 소스 (5가지 소스 타입 기반)
- 🍽️ **menu_composition** (메뉴 구성)
  - 제공 메뉴 카테고리 및 종류
  - 코스 메뉴 또는 단품 메뉴 구성
  
- 💰 **value_proposition** (가성비/가치)
  - 가격대 및 1인당 평균 비용
  - 가격 대비 품질 및 양
  
- 🎨 **dining_atmosphere** (분위기/경험)
  - 레스토랑 분위기 및 서비스
  - 적합한 방문 목적 (데이트, 가족 모임 등)
  
- 🅿️ **reservation_parking** (예약/주차)
  - 예약 방법 및 난이도
  - 주차 시설 정보
  
- 👨‍🍳 **chef_approach** (셰프 접근법)
  - 셰프 배경 및 요리 스타일
  - 식재료 선택 및 활용 방식

### 프롬프트 아키텍처
- **시스템/유저 프롬프트 분리 패턴** 사용
- **Few-shot Learning**: 10개의 고품질 예시로 학습
- **JSON 출력 강제**: 구조화된 결과 보장

### 출력 형식
```json
{
  "shop_seq": 1,
  "shop_name": "매장명",
  "title": "생면 파스타의 탄탄한 식감과 어란 파스타의 감칠맛 조합",
  "summaries": [
    "지역 + 컨셉 + 타겟 고객층 문장 (40-60자)",
    "시그니처 메뉴 상세 묘사 문장 (40-60자)",
    "공간 특징 + 방문 용도 문장 (40-60자)"
  ]
}
```

## 🚀 사용 워크플로우

```
1. 섹션 1-3: 환경 설정 (최초 1회)
2. 섹션 4: SHOPS 리스트에 매장 정보 입력
3. 섹션 5: 건너뛰기 (섹션 4에서 정보 입력 완료)
4. 섹션 6: 프롬프트 실행 → 여러 매장 일괄 생성
5. 섹션 7: 결과 검증 (선택)
6. 섹션 8: JSON + CSV 파일 저장 (선택)
```

## ⚠️ 섹션 1-3: 최초 1회만 실행 (수정 금지)

In [ ]:
# 섹션 1: Package 설치
%pip install google-cloud-aiplatform google-genai python-dotenv

In [ ]:
# 섹션 2: 라이브러리 Import
import os
import json
import pandas as pd
import vertexai
from google import genai
from google.genai import types
from dotenv import load_dotenv

print("✅ 라이브러리 Import 완료")

In [ ]:
# 섹션 3: Vertex AI 초기화
# 환경 변수 로드
load_dotenv()

# GCP 프로젝트 설정
PROJECT_ID = "wad-dw"  # 실제 GCP 프로젝트 ID (프로젝트 이름 아님!)
LOCATION = "us-central1"

# Vertex AI 초기화
vertexai.init(project=PROJECT_ID, location=LOCATION)
client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

print(f"✅ Vertex AI 초기화 완료: {PROJECT_ID} ({LOCATION})")

## ✅ 섹션 4: 매장 정보 입력 (수정 가능)

### 📋 입력 방법

1. **SHOPS 리스트**: 매장 기본 정보 (shop_seq, shop_name)
2. **COLLECTED_INFO 딕셔너리**: 매장별 수집된 정보를 매장명을 키로 입력

In [ ]:
# ========================================
# 매장 정보 입력
# ========================================

# 1️⃣ 매장 기본 정보 (shop_seq, shop_name)
SHOPS = [
    (1, "레스토랑 우오보"),
    (2, "퍼멘츠"),
    (3, "스시소라 정자점"),
    (4, "미트컬쳐"),
    (5, "그리노 성수"),
    (6, "밴건디스테이크하우스"),
    (7, "몽중헌 청담점"),
    (8, "더기와 합정점"),
    (9, "DOTZ"),
    (10, "야키토리 해공"),
]

# 2️⃣ 매장별 수집된 정보 (매장명을 키로 사용)
COLLECTED_INFO = {
    "레스토랑 우오보": "",
    "퍼멘츠": "",
    "스시소라 정자점": "",
    "미트컬쳐": "",
    "그리노 성수": "",
    "밴건디스테이크하우스": "",
    "몽중헌 청담점": "",
    "더기와 합정점": "",
    "DOTZ": "",
    "야키토리 해공": "",
}

# 3️⃣ SHOP_LIST 자동 생성 (shop_seq, shop_name, collected_info 매핑)
SHOP_LIST = [
    {
        "shop_seq": seq,
        "shop_name": name,
        "collected_info": COLLECTED_INFO.get(name, "[정보 없음 - COLLECTED_INFO에 추가 필요]")
    }
    for seq, name in SHOPS
]

# 출력
print(f"📍 다중 매장 배치 모드")
print(f"   - 총 매장 수: {len(SHOP_LIST)}개\n")
print("=" * 80)
for idx, shop in enumerate(SHOP_LIST, 1):
    info_length = len(shop['collected_info'].strip())
    status = "✅" if info_length > 100 else "⚠️"
    print(f"{status} [{idx}] {shop['shop_name']} (seq: {shop['shop_seq']}) - {info_length:,}자")
print("=" * 80)

if any(len(shop['collected_info'].strip()) < 100 for shop in SHOP_LIST):
    print("\n⚠️ 일부 매장의 정보가 부족합니다. COLLECTED_INFO를 확인하세요.")

## ✅ 섹션 5: 소스 타입별 정보 구조 안내

### 📂 중저가 예약 매장 소스 타입 (5가지)

섹션 4의 `COLLECTED_INFO`에 입력할 정보는 아래 5가지 소스 타입을 포함해야 합니다:

#### 1️⃣ menu_composition (메뉴 구성)
- 제공하는 메뉴 카테고리 및 종류
- 코스 메뉴 또는 단품 메뉴 구성
- 계절 메뉴 및 메뉴 변경 주기

#### 2️⃣ value_proposition (가성비/가치)
- 가격대 및 1인당 평균 비용
- 가격 대비 품질 및 양
- 특별 혜택이나 프로모션

#### 3️⃣ dining_atmosphere (분위기/경험)
- 레스토랑 분위기 (캐주얼, 격식, 로맨틱 등)
- 적합한 방문 목적 (데이트, 가족 모임, 비즈니스 등)
- 서비스 수준 및 특징

#### 4️⃣ reservation_parking (예약/주차)
- 예약 방법 및 난이도
- 예약 필수 여부
- 주차 시설 (발렛, 자체 주차장, 주변 주차장)

#### 5️⃣ chef_approach (셰프 접근법)
- 셰프의 배경 및 경력 (간략히)
- 요리 스타일 및 특징
- 식재료 선택 및 활용 방식

### 💡 정보 입력 예시

```python
COLLECTED_INFO = {
    "레스토랑 우오보": """
    ## MENU_COMPOSITION
    - 생면 파스타 4종, 리조또 2종
    - 코스 메뉴: 3코스 (전채 + 파스타 + 디저트)
    - 계절별 신메뉴 출시
    
    ## VALUE_PROPOSITION
    - 1인당 평균 2만원대
    - 파스타 1.5인분 푸짐한 양
    - 런치 세트 메뉴 할인 (20% 할인)
    
    ## DINING_ATMOSPHERE
    - 모던하고 깔끔한 인테리어
    - 데이트 및 소모임에 적합
    - 친절한 서비스와 메뉴 설명
    
    ## RESERVATION_PARKING
    - 네이버 예약 또는 전화 예약
    - 주말 예약 권장 (1주 전)
    - 건물 지하 주차장 이용 가능
    
    ## CHEF_APPROACH
    - 이탈리아 유학 3년 경력
    - 생면 직접 제조
    - 로컬 식재료 우선 사용
    """
}
```

⚠️ **주의**: 이 섹션은 정보 구조를 안내하는 용도입니다. 실제 정보는 섹션 4의 `COLLECTED_INFO`에 입력하세요.

In [ ]:
# ========================================
# 정보 입력 확인
# ========================================

print("📥 매장별 정보 입력 현황\n")
print("=" * 80)

for idx, shop in enumerate(SHOP_LIST, 1):
    shop_name = shop['shop_name']
    collected_info = shop['collected_info']
    info_length = len(collected_info.strip())
    
    # 상태 표시
    if info_length < 50:
        status = "❌ 정보 없음"
    elif info_length < 200:
        status = "⚠️ 정보 부족"
    else:
        status = "✅ 정보 충분"
    
    print(f"[{idx}] {shop_name}")
    print(f"    상태: {status} ({info_length:,}자)")
    print(f"    권장: 각 소스 타입당 80~400자 (총 400~2000자)")
    print()

print("=" * 80)
print("\n⚠️ 정보가 부족한 매장은 섹션 4의 COLLECTED_INFO를 수정하세요.")
print("➡️ 정보 입력이 완료되었으면 섹션 6을 실행하세요.")

## ✅ 섹션 6: 프롬프트 작성 및 요약문 생성 (수정 가능)

### 프롬프트 구조 설계

이 노트북은 **시스템/유저 프롬프트 분리 패턴**을 사용합니다:

#### 📌 시스템 프롬프트 (System Prompt)
- **역할**: AI의 정체성, 규칙, 출력 형식 정의
- **특징**: 고정된 내용 (매장마다 동일)
- **내용**: 중저가 예약 매장 작성 원칙, 구조 규칙, 출력 형식

#### 📌 유저 프롬프트 (User Prompt)
- **역할**: 구체적인 작업 요청 및 데이터 제공
- **특징**: 매장마다 변경되는 내용
- **내용**: 매장 정보, 수집된 정보, Few-shot 예시

#### 🔗 프롬프트 결합
Google Gemini는 시스템/유저를 API 레벨에서 구분하지 않으므로, 전송 시 하나의 문자열로 결합합니다.

### 프롬프트 작성 규칙
- ❌ **f-string 사용 금지** (변수가 아직 정의되지 않아 에러)
- ✅ **일반 문자열 + `.format()` 사용**
- ✅ **JSON 예시에서 `{{` `}}` 사용** (이스케이프)

In [ ]:
# ========================================
# 프롬프트 정의
# ========================================

# -----------------------------------------
# [시스템 프롬프트] AI의 정체성과 규칙
# -----------------------------------------
system_prompt = """
당신은 중저가 예약 매장 전문 작가입니다.
리뷰 데이터 없이 외부 정보만으로 고객이 실용적으로 활용할 수 있는 매장 요약문을 작성합니다.

## 작성 원칙
1. **실용성**: 누가, 언제, 왜 가야 하는지 명확히 전달
2. **구체성**: 메뉴명, 지역명, 가격대를 구체적으로 명시
3. **직관성**: 식감과 맛을 직관적으로 표현 (과장 금지)
4. **친근함**: 격식보다 친근하고 솔직한 톤
5. **가이드성**: 방문 용도를 명확히 제시 (데이트, 기념일 등)
6. **독창성**: 예시를 그대로 복사하지 말고, 톤앤매너만 참고하여 해당 매장만의 고유한 특징을 작성

## 소스 타입별 정보 활용 가이드

수집된 정보는 5가지 소스 타입으로 구성됩니다:

1. **MENU_COMPOSITION** (메뉴 구성)
   - 제공 메뉴 카테고리, 코스/단품 구성
   - 계절 메뉴 및 메뉴 특징
   → 주로 **2번 문장**(시그니처 메뉴)과 **제목**에 활용

2. **VALUE_PROPOSITION** (가성비/가치)
   - 가격대, 1인당 평균 비용
   - 가격 대비 품질 및 양
   → 주로 **1번 문장**(지역 + 컨셉 + 가격 밸류)에 활용

3. **DINING_ATMOSPHERE** (분위기/경험)
   - 레스토랑 분위기, 방문 목적
   - 서비스 수준 및 특징
   → 주로 **3번 문장**(공간 특징 + 방문 용도)에 활용

4. **RESERVATION_PARKING** (예약/주차)
   - 예약 방법 및 난이도
   - 주차 시설 정보
   → 필요시 **3번 문장**에 보조 정보로 활용

5. **CHEF_APPROACH** (셰프 접근법)
   - 셰프 배경, 요리 스타일
   - 식재료 선택 및 활용
   → 필요시 **1번 또는 2번 문장**에 보조 정보로 활용

## 요약 문장 구조
- **1번 문장**: 지역 + 핵심 컨셉 + 타겟 고객층 + 평가/가격 밸류
  - VALUE_PROPOSITION + MENU_COMPOSITION 정보 활용
- **2번 문장**: 시그니처 메뉴 2개 + 식감/맛/조합 상세 묘사
  - MENU_COMPOSITION 정보 중심 활용
- **3번 문장**: 공간 특징 + 구체적 방문 용도
  - DINING_ATMOSPHERE + RESERVATION_PARKING 정보 활용

## 제목 작성 규칙
- 형식: "[시그니처 메뉴명]의 [식감/맛 형용사]와 [특징 요소] 조합"
- 예시: "생면 파스타의 탄탄한 식감과 어란 파스타의 감칠맛 조합"
- 길이: 20-35자
- MENU_COMPOSITION 정보 활용

## 중저가 예약 매장 특화 가이드
- ✅ 지역명 명시 (양재천, 분당, 성수동 등)
- ✅ 타겟 고객층 명시 (이탈리안 애호가, 여성층, 젊은 층)
- ✅ 가격 언급 (엔트리급, 합리적, 가성비)
- ✅ 화제성 강조 (수요미식회, 흑백 요리사, SNS)
- ✅ 메뉴 조합 상세 (라즈베리 잼 + 매쉬포테이토)
- ✅ 식감 직관적 표현 (촉촉한, 사르르 녹는, 탄탄한)
- ✅ 방문 용도 명확히 (데이트 장소로, 기념일 식사로)
- ❌ 철학/기법 중심 표현 금지
- ❌ 과도한 전문 용어 사용 금지

## 출력 형식
반드시 유효한 JSON 형식으로만 출력하세요. 다른 설명이나 주석은 포함하지 마세요.
{{
  "shop_seq": 숫자,
  "shop_name": "매장명",
  "title": "제목",
  "summaries": ["문장1", "문장2", "문장3"]
}}
"""

# -----------------------------------------
# [유저 프롬프트 템플릿] 구체적 작업 내용
# -----------------------------------------
user_prompt_template = """
[매장 정보]
매장명: {shop_name}
Shop Seq: {shop_seq}

[수집된 정보 - 소스 타입별]
{collected_info}

[참고 예시 - 아래 예시들의 톤앤매너와 문체만 참고하세요. 내용은 절대 복사하지 마세요.]

**중요**: 아래 예시는 작성 스타일을 보여주기 위한 참고용입니다. 예시의 내용을 그대로 사용하지 말고, 
수집된 정보를 바탕으로 해당 매장만의 고유한 특징을 작성하세요.

---

**예시 1: 레스토랑 우오보**
{{
  "title": "서울 3대 파스타 바의 밀도 높은 수제 생면",
  "summaries": [
    "레스토랑 우오보는 서울 3대 생면 파스타 맛집으로 계란 노른자와 밀가루만으로 직접 반죽한 밀도 높은 수제 생면을 선보이는 전문 파스타 바입니다. '우오보'는 '계란'을 뜻하며, 재료의 본질에 충실한 조리법이 특징입니다.",
    "시그니처 메뉴인 라자냐를 비롯해 파케리 등 다양한 메뉴와 와인 페어링을 즐기기 좋습니다.",
    "양재천 거리에 위치하여 접근성이 좋고, 세련되고 깔끔한 인테리어를 갖추어, 테라스 앞 창가 자리에서는 양재천 뷰를 즐기며 분위기 있는 식사를 할 수 있습니다."
  ]
}}

**예시 2: 퍼멘츠**
{{
  "title": "발효 비건 요리와 음악이 공존하는 문화 공간",
  "summaries": [
    "퍼멘츠는 발효 비건 음식, 핸드메이드 콤부차, 내추럴 와인을 제공하는 발효 비건 키친 & 바입니다. 건강하고 지속가능한 식사를 지향하며, 음악, 서적 등 다양한 문화 콘텐츠가 공존하는 복합 공간으로 알려져 있습니다.",
    "시그니처인 갈릭 머쉬룸 피자, 구운 대파 후무스 등 조미료를 최소화한 담백한 발효식을 선보입니다. 음식과 음악의 밸런스에 대한 호평이 이어지는 곳입니다.",
    "우드톤 인테리어와 식물들로 채워진 자연 친화적인 분위기가 특징입니다."
  ]
}}

**예시 3: 스시소라 정자점**
{{
  "title": "합리적인 가격으로 즐기는 도심 속 모던 오마카세",
  "summaries": [
    "스시소라 정자점은 합리적인 가격대로 이용 가능한 캐주얼 오마카세입니다. 모던하고 깔끔한 분위기로 꾸며져 쾌적하고 편안한 식사를 즐길 수 있으며, 가격 대비 무난하고 괜찮은 수준의 구성에 대한 긍정적인 평가가 이어집니다.",
    "특히 저녁보다 점심이 더 가성비 있으며, 한국인 입맛에 맞게 각색된 요리와 특색 있는 크리미한 와사비가 특징입니다.",
    "지하철 수인분당선 정자역 3번 출구에서 도보 약 3분 거리에 위치합니다."
  ]
}}

**예시 4: 미트컬쳐**
{{
  "title": "안목 해변의 제철 해산물로 구현한 유럽의 맛",
  "summaries": [
    "미트컬쳐는 강릉 안목해변에 위치하며, 신선한 제철 현지 재료를 이용해 유럽식 음식을 선보이는 유럽풍 레스토랑입니다. 한국에서 가장 맛있는 피시 앤 칩스 등 높은 퀄리티의 음식을 합리적인 가격에 즐길 수 있는 곳으로 명성을 얻었습니다.",
    "시그니처인 뉴욕스트립 스테이크를 비롯해 촉촉한 스웨디시 미트볼은 라즈베리 잼, 오이, 매쉬 포테이토와 함께 단짠의 조화를 이루는 조합으로 가장 유명하며, 강릉 해산물을 서양식으로 풀어낸 메뉴들이 돋보입니다.",
    "예약제로 운영되므로 방문 전 예약이 필수입니다. 매장 앞 주차 및 근처 안목해변 공용주차장 이용이 가능합니다."
  ]
}}

**예시 5: 그리노 성수**
{{
  "title": "가성비 좋은 가격대의 깊고 진한 풍미의 이탈리안",
  "summaries": [
    "그리노 성수는 가성비 좋은 가격대에 수준 높은 파스타와 스테이크를 즐길 수 있는 이탈리안 레스토랑입니다. 셰프가 직접 수제로 만드는 요리의 깊고 진한 풍미와 정성에 대한 호평이 이어지며, 데이트나 혼밥 메뉴로 인기가 높습니다.",
    "시그니처 메뉴인 옥수수 먹물 뇨끼는 크리미한 소스에 버터향이 더해져 풍부한 맛을 느낄 수 있으며, 화이트 라구 파스타는 미트가 듬뿍 들어있고 트러플 향이 나는 조합으로 미식적 만족도가 높은 메뉴입니다.",
    "오픈 키친을 통해 요리 과정을 볼 수 있으며, 깔끔하고 세련된 인테리어가 특징입니다. 지하철 2호선 성수역에서 도보 약 10분 거리에 위치합니다."
  ]
}}

**예시 6: 밴건디스테이크하우스**
{{
  "title": "육즙 가득한 프라임 등심과 안심의 정통 스테이크",
  "summaries": [
    "밴건디스테이크하우스는 육즙이 풍부하고 부드러운 식감의 스테이크를 선보이는 서래마을의 정통 스테이크 전문점입니다. 프라임 등급의 채끝 등심, 안심 스테이크에 대한 지역 주민의 호평이 이어집니다.",
    "안심 부위가 부드러운 식감을 자랑하며, 로메인 샐러드가 입맛을 돋우는 시그니처 조합입니다. 특히 스타터, 파스타, 디저트가 포함된 세트 메뉴가 인기가 높습니다.",
    "운치 있는 외관과 여유 있는 테이블 배치로 고급스럽고 편안한 분위기를 제공하여, 기념일이나 가족 모임에 적합합니다."
  ]
}}

**예시 7: 몽중헌 청담점**
{{
  "title": "중국 현지 명인의 솜씨가 돋보이는 프리미엄 딤섬",
  "summaries": [
    "몽중헌 청담점은 진한 중식 요리와 함께 고급스러운 딤섬을 선보이는 프리미엄 중식당입니다. 수요미식회에서도 소개될 만큼 청담동 딤섬 맛집으로 꼽힙니다.",
    "시그니처 딤섬은 구채교(부추 딤섬)와 하교(새우 딤섬)이며, 게살 요리, 관자 요리 등 다양한 코스와 함께 딤섬과 짬뽕을 곁들이는 조합이 인기입니다.",
    "잔잔한 조명과 넓은 테이블이 인상적이며, 룸 공간이 많아 가족 모임이나 비즈니스 식사에 최적화된 분위기입니다. 지하철 수인분당선 압구정로데오역 4번 출구에서 도보 약 15분 거리에 위치합니다."
  ]
}}

**예시 8: 더기와 합정점**
{{
  "title": "기와집 감성과 트렌디한 한식 안주의 완벽한 페어링",
  "summaries": [
    "더기와 합정점은 기와집 외관에 네온사인을 더해 레트로와 모던이 조화된 분위기를 선보이는 퓨전 한식 주점입니다. 술 페어링을 고려한 메뉴 구성과 분위기, 위치 삼박자를 모두 만족시키는 합정의 핫플로 손꼽힙니다.",
    "시그니처 메뉴인 보쌈 칼비빔면은 육즙을 머금은 보쌈과 비빔장의 조화가 훌륭하며, 말차 막걸리와 함께 즐기는 조합이 추천됩니다.",
    "홀과 테라스, 단독 룸이 마련되어 모임 성격에 따라 자리를 고르기 좋습니다. 지하철 2·6호선 합정역에서 도보 약 2분 거리에 위치합니다."
  ]
}}

**예시 9: DOTZ**
{{
  "title": "아시아 각국 요소를 융합한 독창적 퓨전 아시안 다이닝",
  "summaries": [
    "DOTZ(다츠)는 일식, 한식, 중식 등 아시아 각국의 요소를 융합하여 독창적인 메뉴를 선보이는 퓨전 아시안 다이닝 공간입니다. 정식당 수셰프 출신 셰프의 참여로 높은 완성도와 재미있는 메뉴 구성에 대한 미식가들의 긍정적인 평가가 이어집니다.",
    "시그니처 메뉴인 카츠산도는 육즙이 살아있는 두툼한 돈가스와 부드러운 식빵의 조화가 일품입니다. 싱가폴 스타일 치킨 라이스 등 아시안 브런치도 인기가 높습니다.",
    "모던하고 심플한 인테리어가 돋보이며, 지하철 6호선 한강진역 2번 출구에서 도보 약 8분 거리에 위치합니다."
  ]
}}

**예시 10: 야키토리 해공**
{{
  "title": "토종닭 특수 부위를 정통 방식으로 구워낸 프리미엄 야키토리",
  "summaries": [
    "야키토리 해공은 토종닭의 다양한 특수 부위를 중심으로 제공하는 프리미엄 야키토리 오마카세 전문점입니다. 정통 조리법으로 야키토리의 맛과 다양성에 대한 극찬이 이어지며, 부산을 대표하는 고급 야키토리 맛집으로 꼽힙니다.",
    "쫄깃한 식감의 토종닭 특수 부위 코스가 메인이며, 부산 꼼장어 꼬치 구이와 같이 지역 특색을 살린 메뉴를 짚불에 구워 강렬한 향과 탱글탱글한 식감을 구현하는 것이 특징입니다.",
    "세련되고 조용한 분위기의 실내 디자인이 돋보이며, 혼자서도 편하게 즐길 수 있는 캐주얼하면서도 고급스러운 공간입니다."
  ]
}}

---

**다시 한 번 강조**: 위 예시는 작성 스타일(톤, 문체, 구조)을 참고하기 위한 것입니다.
예시의 구체적인 내용(메뉴명, 지역명, 특징 등)을 그대로 사용하지 말고,
[수집된 정보]를 바탕으로 [{shop_name}] 매장만의 고유한 특징을 작성하세요.
"""

print("✅ 프롬프트 템플릿 작성 완료")
print("   - 시스템 프롬프트: 중저가 예약 매장 작성 원칙 + 소스 타입별 매핑 가이드")
print("   - 유저 프롬프트: 구체적 작업 내용")
print("   - Few-shot 예시: 10개 (톤앤매너 참고용)")

In [ ]:
# ========================================
# 모델 설정 및 LLM 호출
# ========================================

# -----------------------------------------
# 모델 파라미터 설정
# -----------------------------------------
MODEL_NAME = "gemini-2.5-pro"  # 높은 품질
TEMPERATURE = 0.5  # 낮은 창의성, 일관성 우선
MAX_OUTPUT_TOKENS = 4096  # JSON 응답을 위한 충분한 토큰

generation_config = types.GenerateContentConfig(
    temperature=TEMPERATURE,
    max_output_tokens=MAX_OUTPUT_TOKENS,
    response_mime_type="application/json"  # JSON 출력 강제
)

print(f"🤖 모델: {MODEL_NAME}")
print(f"   - Temperature: {TEMPERATURE}")
print(f"   - Max Output Tokens: {MAX_OUTPUT_TOKENS:,}\n")

# -----------------------------------------
# 요약문 생성 함수
# -----------------------------------------
def generate_summary(shop_seq, shop_name, collected_info):
    """단일 매장에 대한 요약문 생성"""
    
    # 유저 프롬프트 생성
    user_prompt = user_prompt_template.format(
        shop_seq=shop_seq,
        shop_name=shop_name,
        collected_info=collected_info
    )
    
    # 프롬프트 결합
    combined_prompt = f"{system_prompt}\n\n{user_prompt}"
    
    try:
        # LLM 호출
        response = client.models.generate_content(
            model=MODEL_NAME,
            contents=combined_prompt,
            config=generation_config
        )
        
        # JSON 파싱
        result_text = response.text
        result_json = json.loads(result_text)
        
        # 토큰 사용량 추출
        usage_metadata = response.usage_metadata if hasattr(response, 'usage_metadata') else None
        token_info = {}
        if usage_metadata:
            token_info = {
                "input_tokens": usage_metadata.prompt_token_count,
                "output_tokens": usage_metadata.candidates_token_count,
                "total_tokens": usage_metadata.total_token_count
            }
        
        return {
            "status": "success",
            "data": result_json,
            "tokens": token_info
        }
        
    except json.JSONDecodeError as e:
        return {
            "status": "error",
            "error_type": "JSONDecodeError",
            "message": str(e),
            "response": response.text if 'response' in locals() else None,
            "tokens": {}
        }
        
    except Exception as e:
        return {
            "status": "error",
            "error_type": type(e).__name__,
            "message": str(e),
            "response": response.text if 'response' in locals() else None,
            "tokens": {}
        }

# -----------------------------------------
# 다중 매장 배치 처리
# -----------------------------------------
all_results = []
total_input_tokens = 0
total_output_tokens = 0
total_tokens = 0

print(f"📝 {len(SHOP_LIST)}개 매장 요약문 생성 시작\n")
print("=" * 80)

for idx, shop in enumerate(SHOP_LIST, 1):
    shop_seq = shop["shop_seq"]
    shop_name = shop["shop_name"]
    collected_info = shop["collected_info"]
    
    print(f"\n[{idx}/{len(SHOP_LIST)}] {shop_name} 처리 중...")
    
    result = generate_summary(shop_seq, shop_name, collected_info)
    
    if result["status"] == "success":
        result_json = result["data"]
        all_results.append(result_json)
        
        # 토큰 사용량 출력 및 누적
        tokens = result.get("tokens", {})
        if tokens:
            input_tokens = tokens.get('input_tokens', 0)
            output_tokens = tokens.get('output_tokens', 0)
            total_tokens_shop = tokens.get('total_tokens', 0)
            
            total_input_tokens += input_tokens
            total_output_tokens += output_tokens
            total_tokens += total_tokens_shop
            
            print(f"   ✅ 성공 | Input: {input_tokens:,} | Output: {output_tokens:,} | Total: {total_tokens_shop:,} tokens")
        else:
            print(f"   ✅ 성공")
    else:
        print(f"   ❌ 실패: {result['error_type']} - {result['message']}")
        # 실패한 경우에도 기록
        all_results.append({
            "shop_seq": shop_seq,
            "shop_name": shop_name,
            "status": "error",
            "error": result['message']
        })

print("\n" + "=" * 80)
print(f"\n✅ 전체 완료: {len(all_results)}개 매장 처리")

# 성공/실패 개수
success_count = sum(1 for r in all_results if r.get("status") != "error")
error_count = len(all_results) - success_count
print(f"   - 성공: {success_count}개")
if error_count > 0:
    print(f"   - 실패: {error_count}개")

# 총 토큰 사용량 출력
if total_tokens > 0:
    print(f"\n📊 총 토큰 사용량:")
    print(f"   - Total Input:  {total_input_tokens:,} tokens")
    print(f"   - Total Output: {total_output_tokens:,} tokens")
    print(f"   - Total:        {total_tokens:,} tokens")

# -----------------------------------------
# 전체 결과 상세 출력
# -----------------------------------------
print("\n" + "=" * 80)
print("📋 전체 결과 상세보기")
print("=" * 80)

for idx, result in enumerate(all_results, 1):
    print(f"\n{'─' * 80}")
    print(f"[{idx}] {result.get('shop_name')}")
    print(f"{'─' * 80}")
    
    if result.get("status") != "error":
        # 성공한 경우 - 전체 내용 출력
        print(f"\n📌 제목:")
        print(f"   {result.get('title')}")
        
        print(f"\n📝 요약문:")
        summaries = result.get('summaries', [])
        for i, summary in enumerate(summaries, 1):
            print(f"   {i}. {summary}")
        
        print(f"\n✅ 상태: 성공")
    else:
        # 실패한 경우
        print(f"\n❌ 상태: 에러")
        print(f"   오류 내용: {result.get('error', 'N/A')}")

print("\n" + "=" * 80)

## ✅ 섹션 7: 후처리 검증 (선택 사항)

### 검증 항목

이 섹션은 생성된 요약문의 품질을 **코드 기반**으로 검증합니다 (LLM 호출 없음):

#### 📋 기본 구조 검증
- 필수 키 존재 확인 (shop_seq, shop_name, title, summaries)
- 요약문 개수 (정확히 3개)
- 문장 길이 (40-60자 권장, 30자 미만/100자 초과 경고)
- 제목 길이 (20-35자 권장)

#### 🚫 품질 검증
- 예시 복사 의심 (예시와 90% 이상 유사)
- 과장된 표현 금지 (최고의, 완벽한, 최상의 등)
- 문장 간 중복 내용 (3개 문장이 너무 유사)
- 특수문자/이모지 체크

#### 💡 검증 결과
- ✅ 통과: 모든 규칙 준수
- ⚠️ 경고: 일부 권장사항 위반 (사용 가능하지만 검토 권장)
- ❌ 실패: 필수 규칙 위반 (수정 필요)

In [ ]:
# ========================================
# 강력한 후처리 검증 함수
# ========================================

import difflib
import re

# -----------------------------------------
# 예시 문장 리스트 (복사 의심 체크용)
# -----------------------------------------
EXAMPLE_SENTENCES = [
    "양재천 인근에서 탄탄한 식감의 생면 파스타를 내세워 이탈리안 미식 애호가들 사이에서 높은 평가를 받습니다. 파스타의 간과 소스 밸런스가 훌륭하다는 것이 공통된 의견입니다.",
    "시그니처인 감자 뇨끼는 완벽한 질감으로 유명하며, 어란 파스타는 홍합살과 먹물 스파게티면이 어우러져 감칠맛이 폭발하는 조합으로 높은 추천을 받습니다.",
    "실내는 깔끔하고 세련된 인테리어로 설계되어 있으며, 조용하고 격식 있는 데이트 장소로 꾸준히 화제성을 유지하는 분위기를 제공합니다.",
    "비건(Vegan)과 발효라는 트렌드를 선도하며, 채식 미식가들뿐 아니라 육식파 사이에서도 높은 완성도와 맛을 인정받아 압도적인 화제성을 구축했습니다.",
    "시그니처인 촉촉한 스웨디시 미트볼은 라즈베리 잼, 오이, 매쉬 포테이토와 함께 단짠의 조화를 이루는 조합으로 가장 유명합니다.",
    "성수동 특유의 아기자기하고 감각적인 분위기를 갖춘 브런치 및 파스타 맛집으로, 크리미한 소스가 깊고 진한 풍미를 선사하여 여성 고객층 사이에서 높은 화제성을 보유하고 있습니다.",
]

# -----------------------------------------
# 금지 키워드 리스트 (과장된 표현)
# -----------------------------------------
BANNED_KEYWORDS = [
    "최고의", "최상의", "완벽한", "최고급의", "세계 최고",
    "압도적인", "독보적인", "타의 추종을 불허하는",
    "놀라운", "경이로운", "환상적인", "굉장한"
]

def calculate_similarity(text1, text2):
    """두 문자열 간 유사도 계산 (0.0 ~ 1.0)"""
    return difflib.SequenceMatcher(None, text1, text2).ratio()

def check_example_similarity(sentence):
    """예시 문장과의 유사도 체크"""
    max_similarity = 0.0
    most_similar_example = None
    
    for example in EXAMPLE_SENTENCES:
        similarity = calculate_similarity(sentence, example)
        if similarity > max_similarity:
            max_similarity = similarity
            most_similar_example = example
    
    return max_similarity, most_similar_example

def check_banned_keywords(text):
    """금지 키워드 체크"""
    found_keywords = []
    for keyword in BANNED_KEYWORDS:
        if keyword in text:
            found_keywords.append(keyword)
    return found_keywords

def check_special_chars(text):
    """특수문자/이모지 체크"""
    # 이모지 패턴
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # 이모티콘
        "\U0001F300-\U0001F5FF"  # 기호 & 픽토그램
        "\U0001F680-\U0001F6FF"  # 교통 & 지도
        "\U0001F1E0-\U0001F1FF"  # 국기
        "]+", 
        flags=re.UNICODE
    )
    
    emojis = emoji_pattern.findall(text)
    return emojis

def validate_summary(result, strict=False):
    """
    요약문 검증 함수
    
    Args:
        result: 검증할 결과 dict
        strict: True이면 경고도 실패로 간주
    
    Returns:
        dict: {
            "passed": bool,
            "errors": list,
            "warnings": list
        }
    """
    errors = []
    warnings = []
    
    # -----------------------------------------
    # 1. 필수 키 존재 확인
    # -----------------------------------------
    required_keys = ["shop_seq", "shop_name", "title", "summaries"]
    for key in required_keys:
        if key not in result:
            errors.append(f"❌ 필수 키 누락: {key}")
    
    if errors:  # 필수 키가 없으면 더 이상 검증 불가
        return {
            "passed": False,
            "errors": errors,
            "warnings": warnings
        }
    
    # -----------------------------------------
    # 2. 요약문 개수 확인
    # -----------------------------------------
    summaries = result.get("summaries", [])
    if len(summaries) != 3:
        errors.append(f"❌ 요약문 개수 오류: {len(summaries)}개 (3개 필요)")
        return {
            "passed": False,
            "errors": errors,
            "warnings": warnings
        }
    
    # -----------------------------------------
    # 3. 문장 길이 확인
    # -----------------------------------------
    for i, sentence in enumerate(summaries, 1):
        length = len(sentence)
        if length < 30:
            warnings.append(f"⚠️ {i}번 문장이 너무 짧음: {length}자 (권장 40-60자)")
        elif length > 100:
            warnings.append(f"⚠️ {i}번 문장이 너무 김: {length}자 (권장 40-60자)")
    
    # -----------------------------------------
    # 4. 제목 길이 확인
    # -----------------------------------------
    title = result.get("title", "")
    title_length = len(title)
    if title_length < 15:
        warnings.append(f"⚠️ 제목이 너무 짧음: {title_length}자 (권장 20-35자)")
    elif title_length > 45:
        warnings.append(f"⚠️ 제목이 너무 김: {title_length}자 (권장 20-35자)")
    
    # -----------------------------------------
    # 5. 예시 복사 의심 체크
    # -----------------------------------------
    for i, sentence in enumerate(summaries, 1):
        similarity, similar_example = check_example_similarity(sentence)
        if similarity >= 0.9:
            errors.append(f"❌ {i}번 문장이 예시를 복사한 것으로 의심됨 (유사도 {similarity:.1%})")
            errors.append(f"   원본: {similar_example[:50]}...")
        elif similarity >= 0.7:
            warnings.append(f"⚠️ {i}번 문장이 예시와 유사함 (유사도 {similarity:.1%})")
    
    # -----------------------------------------
    # 6. 금지 키워드 체크
    # -----------------------------------------
    all_text = title + " ".join(summaries)
    banned_found = check_banned_keywords(all_text)
    if banned_found:
        warnings.append(f"⚠️ 과장된 표현 발견: {', '.join(banned_found)}")
    
    # -----------------------------------------
    # 7. 특수문자/이모지 체크
    # -----------------------------------------
    emojis = check_special_chars(all_text)
    if emojis:
        warnings.append(f"⚠️ 이모지 발견: {', '.join(emojis)}")
    
    # -----------------------------------------
    # 8. 문장 간 중복 체크 (3개 문장이 너무 유사)
    # -----------------------------------------
    for i in range(len(summaries)):
        for j in range(i+1, len(summaries)):
            similarity = calculate_similarity(summaries[i], summaries[j])
            if similarity >= 0.7:
                warnings.append(f"⚠️ {i+1}번과 {j+1}번 문장이 너무 유사함 (유사도 {similarity:.1%})")
    
    # -----------------------------------------
    # 결과 판정
    # -----------------------------------------
    if strict:
        passed = len(errors) == 0 and len(warnings) == 0
    else:
        passed = len(errors) == 0
    
    return {
        "passed": passed,
        "errors": errors,
        "warnings": warnings
    }

# -----------------------------------------
# 전체 결과 검증 실행
# -----------------------------------------
if len(all_results) == 0:
    print("⚠️ 검증할 결과가 없습니다.")
    print("   먼저 섹션 6을 실행하여 요약문을 생성하세요.")
else:
    print("🔍 요약문 검증 시작\n")
    print("=" * 80)
    
    total_passed = 0
    total_warnings = 0
    total_failed = 0
    
    validation_results = []
    
    for idx, result in enumerate(all_results, 1):
        # 에러 상태인 결과는 스킵
        if result.get("status") == "error":
            print(f"\n[{idx}] {result.get('shop_name')}: 생성 실패로 검증 스킵")
            continue
        
        shop_name = result.get("shop_name", "Unknown")
        print(f"\n[{idx}] {shop_name}")
        print("─" * 80)
        
        # 검증 실행
        validation = validate_summary(result, strict=False)
        validation_results.append({
            "shop_name": shop_name,
            "validation": validation
        })
        
        errors = validation["errors"]
        warnings = validation["warnings"]
        passed = validation["passed"]
        
        if passed and len(warnings) == 0:
            print("✅ 검증 통과: 모든 규칙을 준수합니다")
            total_passed += 1
        elif passed:
            print(f"✅ 검증 통과 (경고 {len(warnings)}개)")
            total_passed += 1
            total_warnings += 1
            for warning in warnings:
                print(f"   {warning}")
        else:
            print(f"❌ 검증 실패 (오류 {len(errors)}개, 경고 {len(warnings)}개)")
            total_failed += 1
            for error in errors:
                print(f"   {error}")
            for warning in warnings:
                print(f"   {warning}")
    
    # -----------------------------------------
    # 전체 검증 요약
    # -----------------------------------------
    print("\n" + "=" * 80)
    print("📊 검증 요약")
    print("=" * 80)
    print(f"✅ 통과: {total_passed}개")
    if total_warnings > 0:
        print(f"⚠️ 경고 있음: {total_warnings}개")
    if total_failed > 0:
        print(f"❌ 실패: {total_failed}개")
    
    success_rate = (total_passed / len([r for r in all_results if r.get("status") != "error"])) * 100
    print(f"\n성공률: {success_rate:.1f}%")

## ✅ 섹션 8: 결과 저장 (선택 사항)

### 저장 형식

#### 단일 매장 모드
- JSON 파일 1개 (전체 구조 포함)
- 파일명: `summary_{매장명}_{타임스탬프}.json`

#### 다중 매장 모드
- JSON 파일 (전체 구조 포함)
- CSV 파일 (분석용 테이블 형식)
- 파일명: `summaries_batch_{타임스탬프}.json/csv`

In [ ]:
# ========================================
# 결과 저장
# ========================================

from datetime import datetime
import pandas as pd

if len(all_results) == 0:
    print("⚠️ 저장할 결과가 없습니다.")
else:
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # JSON 파일
    json_filename = f"summaries_batch_{timestamp}.json"
    with open(json_filename, 'w', encoding='utf-8') as f:
        json.dump(all_results, f, ensure_ascii=False, indent=2)
    
    print(f"✅ JSON 저장 완료: {json_filename}")
    
    # CSV 파일
    csv_data = []
    for result in all_results:
        if result.get("status") != "error":
            csv_data.append({
                "shop_seq": result.get("shop_seq"),
                "shop_name": result.get("shop_name"),
                "title": result.get("title"),
                "summary_1": result.get("summaries", [])[0] if len(result.get("summaries", [])) > 0 else "",
                "summary_2": result.get("summaries", [])[1] if len(result.get("summaries", [])) > 1 else "",
                "summary_3": result.get("summaries", [])[2] if len(result.get("summaries", [])) > 2 else "",
                "status": "success"
            })
    
    df = pd.DataFrame(csv_data)
    csv_filename = f"summaries_batch_{timestamp}.csv"
    df.to_csv(csv_filename, index=False, encoding='utf-8-sig')
    
    print(f"✅ CSV 저장 완료: {csv_filename}")